## Mersenne Circle Group FFT

It demonstrates how to compute the Fast Fourier Transform (FFT) on a Mersenne circle group using Rader’s algorithm. In this context, we focus on sequences whose length is a Mersenne prime (of the form $2^p - 1$). For demonstration purposes, we use $N = 7$ (since $7 = 2^3 - 1$) as our prime length.

### Introduction

In many applications, such as polynomial commitment schemes and cryptographic proof systems, it is necessary to convert between the evaluation representation and the coefficient representation of a polynomial. When the evaluation domain is a circle (often chosen as roots of unity), the Fast Fourier Transform (FFT) offers an efficient solution. In this notebook, we define a circle domain, perform interpolation (recovering polynomial coefficients from evaluations), and show how to evaluate the polynomial at arbitrary points. We also include an example of extrapolation (low-degree extension).

### Defining the Circle Domain

We define a `CircleDomain` class that represents a set of points on the complex unit circle. For simplicity, the domain size is chosen as a power of two.

In [2]:
import numpy as np

class CircleDomain:
    def __init__(self, log_n, shift=0):
        """
        Initialize the circle domain.
        
        Args:
            log_n: logarithm (base 2) of the domain size.
            shift: a phase shift applied to all points.
        """
        self.log_n = log_n
        self.n = 1 << log_n  # Domain size: 2^log_n
        self.shift = shift
        # Generate points on the unit circle with an optional shift.
        self.points = np.array([np.exp(2j * np.pi * (k + shift) / self.n) for k in range(self.n)])
    
    def __iter__(self):
        return iter(self.points)
    
    def size(self):
        return self.n

### Circle Evaluations and Interpolation

The `CircleEvaluations` class encapsulates evaluation values (typically obtained by an FFT) over a circle domain. It provides methods to interpolate (recover polynomial coefficients), evaluate the polynomial at an arbitrary point, and extrapolate to a larger domain.

In [3]:
class CircleEvaluations:
    def __init__(self, domain, values):
        """
        Initialize with a circle domain and corresponding evaluation values.
        
        Args:
            domain: an instance of CircleDomain.
            values: a list or numpy array of evaluation values.
        """
        self.domain = domain
        self.values = np.array(values)
        assert self.values.shape[0] == self.domain.size(), "Mismatch between domain size and number of values."
    
    def interpolate(self):
        """
        Interpolate the polynomial coefficients from the evaluation values using the inverse FFT.
        
        Returns:
            The recovered polynomial coefficients as a numpy array.
        """
        coeffs = np.fft.ifft(self.values)
        return coeffs
    
    def evaluate_at_point(self, point):
        """
        Evaluate the interpolated polynomial at an arbitrary point.
        
        This is achieved by computing the dot product of the polynomial coefficients
        with the monomial basis evaluated at the point.
        
        Args:
            point: the point at which to evaluate the polynomial.
        
        Returns:
            The polynomial evaluation at the given point.
        """
        coeffs = self.interpolate()
        n = len(coeffs)
        powers = np.array([point**i for i in range(n)])
        return np.dot(coeffs, powers)
    
    def extrapolate(self, target_log_n):
        """
        Extrapolate (compute a low-degree extension) by zero-padding the coefficients to a larger domain.
        
        Args:
            target_log_n: logarithm (base 2) of the target domain size (must be larger than the current domain).
        
        Returns:
            A new CircleEvaluations instance with the extrapolated evaluations.
        """
        coeffs = self.interpolate()
        current_n = len(coeffs)
        target_n = 1 << target_log_n
        if target_n < current_n:
            raise ValueError("Target domain must be larger than current domain.")
        # Zero-pad the coefficients.
        padded_coeffs = np.concatenate([coeffs, np.zeros(target_n - current_n, dtype=complex)])
        # Compute the FFT on the padded coefficients to obtain the new evaluation values.
        new_values = np.fft.fft(padded_coeffs)
        new_domain = CircleDomain(target_log_n, shift=self.domain.shift)
        return CircleEvaluations(new_domain, new_values)

We generate a random polynomial, compute its evaluations on a circle domain via FFT, interpolate to recover the coefficients, and verify the evaluation at an arbitrary point.

In [4]:
# Set up a circle domain with log_n = 3 (n = 8)
log_n = 3
domain = CircleDomain(log_n)

# Generate a random polynomial of degree n-1.
np.random.seed(42)
coeffs = np.random.random(1 << log_n) + 1j * np.random.random(1 << log_n)

# Compute evaluations using FFT.
values = np.fft.fft(coeffs)

# Create a CircleEvaluations instance.
circle_evals = CircleEvaluations(domain, values)

# Interpolate to recover the coefficients.
recovered_coeffs = circle_evals.interpolate()
print("Original Coefficients:")
print(coeffs)
print("\nRecovered Coefficients (via interpolation):")
print(recovered_coeffs)

# Evaluate the polynomial at an arbitrary point (e.g., 1.2 + 0.5j).
point = 1.2 + 0.5j
eval_at_point = circle_evals.evaluate_at_point(point)
# Direct evaluation using the recovered coefficients.
direct_eval = sum(c * (point**i) for i, c in enumerate(recovered_coeffs))
print("\nEvaluation at point {}:".format(point))
print("Using evaluate_at_point:", eval_at_point)
print("Direct evaluation:", direct_eval)

Original Coefficients:
[0.37454012+0.60111501j 0.95071431+0.70807258j 0.73199394+0.02058449j
 0.59865848+0.96990985j 0.15601864+0.83244264j 0.15599452+0.21233911j
 0.05808361+0.18182497j 0.86617615+0.18340451j]

Recovered Coefficients (via interpolation):
[0.37454012+0.60111501j 0.95071431+0.70807258j 0.73199394+0.02058449j
 0.59865848+0.96990985j 0.15601864+0.83244264j 0.15599452+0.21233911j
 0.05808361+0.18182497j 0.86617615+0.18340451j]

Evaluation at point (1.2+0.5j):
Using evaluate_at_point: (-9.093525188073993+6.003573307765665j)
Direct evaluation: (-9.093525188073993+6.003573307765664j)


We demonstrate how to extrapolate to a larger domain. This is analogous to computing a low-degree extension (LDE) of the polynomial.

In [5]:
# Extrapolate to a larger domain (e.g., log_n = 4, so n = 16).
target_log_n = 4
extrapolated_evals = circle_evals.extrapolate(target_log_n)

# Interpolate the extrapolated evaluations to obtain new coefficients.
extrapolated_coeffs = extrapolated_evals.interpolate()
print("\nExtrapolated Coefficients (zero-padded):")
print(extrapolated_coeffs)


Extrapolated Coefficients (zero-padded):
[ 3.74540119e-01+6.01115012e-01j  9.50714306e-01+7.08072578e-01j
  7.31993942e-01+2.05844943e-02j  5.98658484e-01+9.69909852e-01j
  1.56018640e-01+8.32442641e-01j  1.55994520e-01+2.12339111e-01j
  5.80836122e-02+1.81824967e-01j  8.66176146e-01+1.83404510e-01j
  2.77555756e-17-5.55111512e-17j  0.00000000e+00+0.00000000e+00j
 -5.55111512e-17-1.38777878e-17j -5.55111512e-17+5.55111512e-17j
  0.00000000e+00+0.00000000e+00j -1.38777878e-16-6.93889390e-17j
 -4.85722573e-17+6.93889390e-17j  0.00000000e+00+1.38777878e-17j]


### Conclusion

In this notebook, we demonstrated a simplified version of FFT-based interpolation and extrapolation on a circle domain. Inspired by a Rust implementation that uses advanced techniques (such as parallel processing and specialized butterfly operations), our Python version shows how to:
- Define a circle domain (representing evaluation points on the unit circle),
- Recover polynomial coefficients from evaluation values using the inverse FFT,
- Evaluate the polynomial at arbitrary points using a monomial basis,
- Extend the domain via zero-padding (extrapolation).

This method forms a foundation for many advanced algorithms in polynomial commitment schemes and cryptographic protocols.